In [1]:
import numpy as np

def simulate_eca(rule_number, initial_state, steps):
    """
    Simulate an Elementary Cellular Automaton (ECA).

    Parameters:
        rule_number (int): The rule number (0-255).
        initial_state (list): Binary array representing the initial state.
        steps (int): Number of steps to simulate.

    Returns:
        np.ndarray: 2D grid of CA states over time.
    """
    # Convert rule number to binary rule set
    rule_binary = np.array([int(x) for x in np.binary_repr(rule_number, width=8)])

    # Initialize the grid
    n = len(initial_state)
    grid = np.zeros((steps, n), dtype=int)
    grid[0] = initial_state

    for t in range(1, steps):
        for i in range(1, n):
            neighborhood = (grid[t - 1, i - 1] << 2) | (grid[t - 1, i] << 1) | grid[t - 1, (i + 1) % n]
            grid[t, i] = rule_binary[7 - neighborhood]

    return grid

# Example usage:
initial_state = [0] * 20 + [1] + [0] * 20
rule_number = 110
steps = 50
grid = simulate_eca(rule_number, initial_state, steps)

In [2]:
def extract_windows(grid, window_size):
    """
    Extract spatiotemporal windows from the ECA grid.

    Parameters:
        grid (np.ndarray): 2D grid of CA states.
        window_size (int): Size of the temporal window.

    Returns:
        list of np.ndarray: Extracted windows.
    """
    windows = []
    for i in range(grid.shape[0] - window_size + 1):
        windows.append(grid[i:i + window_size])
    return np.array(windows)

# Example usage:
window_size = 10
windows = extract_windows(grid, window_size)

In [3]:
def compute_lempel_ziv_complexity(sequence):
    """Calculate Lempel-Ziv complexity."""
    substrings = set()
    complexity = 0
    current = ""
    for char in sequence:
        current += char
        if current not in substrings:
            substrings.add(current)
            complexity += 1
            current = ""
    return complexity

sequence = "".join(map(str, grid.flatten()))
lz_complexity = compute_lempel_ziv_complexity(sequence)
print(f"Lempel-Ziv Complexity: {lz_complexity}")

Lempel-Ziv Complexity: 213


In [4]:
def krylov_complexity(grid):
    """
    Compute the Krylov complexity over time.

    Parameters:
        grid (np.ndarray): 2D array of automaton states.

    Returns:
        list: Krylov complexity values over time.
    """
    seen_states = set()
    complexity = []

    for row in grid:
        seen_states.add(tuple(row))
        complexity.append(len(seen_states))

    return complexity

complexity = krylov_complexity(grid)
print(f"Krylov Complexity over time: {complexity}")

Krylov Complexity over time: [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50]


In [5]:
import torch
from torch import nn
from torch.utils.data import DataLoader, TensorDataset

class SimpleTransformer(nn.Module):
    def __init__(self, input_dim, seq_length, num_heads, hidden_dim, num_layers):
        super().__init__()
        self.embedding = nn.Linear(input_dim, hidden_dim)
        encoder_layer = nn.TransformerEncoderLayer(d_model=hidden_dim, nhead=num_heads)
        self.encoder = nn.TransformerEncoder(encoder_layer, num_layers=num_layers)
        self.output_layer = nn.Linear(hidden_dim, input_dim)

    def forward(self, x):
        x = self.embedding(x)
        x = self.encoder(x)
        return self.output_layer(x)

# Preparing the dataset
train_data = torch.tensor(windows, dtype=torch.float32)
train_loader = DataLoader(TensorDataset(train_data), batch_size=32, shuffle=True)

# Training the model
model = SimpleTransformer(input_dim=grid.shape[1], seq_length=window_size, num_heads=4, hidden_dim=64, num_layers=2)
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)
criterion = nn.CrossEntropyLoss()

for epoch in range(10):
    for batch in train_loader:
        inputs = batch[0][:, :-1]
        targets = batch[0][:, 1:]
        outputs = model(inputs)
        loss = criterion(outputs.view(-1, grid.shape[1]), targets.view(-1))
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

d:\books\cellular-automata-book\.env\Lib\site-packages\torch\nn\modules\transformer.py:286: UserWarning: enable_nested_tensor is True, but self.use_nested_tensor is False because encoder_layer.self_attn.batch_first was not True(use batch_first for better inference performance)
  warnings.warn(f"enable_nested_tensor is True, but self.use_nested_tensor is False because {why_not_sparsity_fast_path}")


RuntimeError: view size is not compatible with input tensor's size and stride (at least one dimension spans across two contiguous subspaces). Use .reshape(...) instead.